In [6]:
import pandas as pd
import numpy as np
import sys
import dash
from dash import Dash, html, dcc, Input, Output, no_update
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from flask_caching import Cache
try:
    from scipy import stats
    HAVE_KDE = True
except Exception:
    HAVE_KDE = False

# =========================
# Load + normalize data
# =========================
try:
    matches_raw = pd.read_csv("../data/cleaned/matches.csv")
    deliveries_raw = pd.read_csv("../data/cleaned/deliveries.csv")
    venue_coords = pd.read_csv("../data/cleaned/venue_coords.csv")
except:
    matches_raw = pd.read_csv("data/matches.csv")
    deliveries_raw = pd.read_csv("data/deliveries.csv")
    venue_coords = pd.read_csv("data/venue_coords.csv")

# Standardize column names
matches_6 = matches_raw.rename(columns={
    "Id":"match_id","Match_Id":"match_id","Season":"season",
    "Team1":"team1","Team2":"team2","Venue":"venue","Date":"date",
    "Winner":"winner","Result":"result","Umpire1":"ump1","Umpire2":"ump2",
    "Time":"Time","Toss_Winner":"toss_winner","Toss_Decision":"Toss_Decision","City":"City"
})

deliveries_6 = deliveries_raw.rename(columns={
    "Match_Id":"match_id","Inning":"inning","Over":"over","Ball":"ball",
    "Batter":"batter","Bowler":"bowler","Batting_Team":"batting_team",
    "Bowling_Team":"bowling_team","Batsman_Runs":"batsman_runs",
    "Total_Runs":"total_runs","Is_Wicket":"is_wicket",
    "Player_Dismissed":"player_dismissed","Extras_Type":"extras_type",
    "Dismissal_Kind":"dismissal_kind"
})

if "is_wicket" not in deliveries_6.columns:
    deliveries_6["is_wicket"] = deliveries_6["player_dismissed"].notna().astype(int)

# Seasons for dropdowns
if "season" in matches_6.columns:
    SEASONS = sorted(pd.to_numeric(matches_6["season"], errors="coerce").dropna().astype(int).unique().tolist())
else:
    SEASONS = []

# =========================
# Theme & Team Colors
# =========================
COLORS = {
    "bg_dark": "#0b1220",
    "card_bg": "#111a2b", 
    "text": "#e5e7eb",
    "accent": "#00e6a8"
}

TEAM_INFO = {
    "Chennai Super Kings": {"abbr": "CSK", "color": "#f1c40f"},
    "Mumbai Indians": {"abbr": "MI", "color": "#004ba0"},
    "Royal Challengers Bangalore": {"abbr": "RCB", "color": "#da291c"},
    "Kolkata Knight Riders": {"abbr": "KKR", "color": "#3b0a45"},
    "Rajasthan Royals": {"abbr": "RR", "color": "#ea1a8e"},
    "Sunrisers Hyderabad": {"abbr": "SRH", "color": "#ff822a"},
    "Delhi Capitals": {"abbr": "DC", "color": "#17449b"},
    "Punjab Kings": {"abbr": "PBKS", "color": "#d71920"},
    "Gujarat Titans": {"abbr": "GT", "color": "#041c2c"},
    "Lucknow Super Giants": {"abbr": "LSG", "color": "#00b2a9"},
    "Gujarat Lions": {"abbr": "GL", "color": "#f26522"},
    "Rising Pune Supergiant": {"abbr": "RPS", "color": "#800080"},
    "Pune Warriors India": {"abbr": "PWI", "color": "#00a3e0"},
    "Kochi Tuskers Kerala": {"abbr": "KTK", "color": "#f26a21"},
    "Deccan Chargers": {"abbr": "DCG", "color": "#1e4fa1"}
}

REGION_COLORS = {"UAE": "#F59E0B", "SA": "#10B981"}

def get_team_abbr(team):
    if not isinstance(team, str): return "UNK"
    for k, v in TEAM_INFO.items():
        if k.lower() in team.lower():
            return v["abbr"]
    return team[:3].upper()

def get_team_color(team):
    if not isinstance(team, str): return "#888888"
    for k, v in TEAM_INFO.items():
        if k.lower() in team.lower():
            return v["color"]
    return "#888888"

def pick_color_for_teamname(teamname_or_abbr):
    if pd.isna(teamname_or_abbr): return "#9e9e9e"
    abbr = teamname_or_abbr if len(teamname_or_abbr) <= 4 and teamname_or_abbr.isupper() else get_team_abbr(teamname_or_abbr)
    if abbr and abbr in [v["abbr"] for v in TEAM_INFO.values()]:
        for team_info in TEAM_INFO.values():
            if team_info["abbr"] == abbr:
                return team_info["color"]
    if teamname_or_abbr in REGION_COLORS:
        return REGION_COLORS[teamname_or_abbr]
    return "#9e9e9e"

# =========================
# Helper Functions
# =========================
def kpi_card_unified(title, value, border="#334155", text_color=None, small=False):
    return html.Div([
        html.Div(title, style={"fontSize": "12px", "color": COLORS["accent"], "opacity": 0.9}),
        html.H4(str(value), style={
            "margin": "0.1rem 0 0 0",
            "color": text_color or COLORS["text"],
            "fontWeight": "700" if not small else "600"
        }),
    ], style={
        "background": COLORS["card_bg"],
        "border": f"1px solid {border}",
        "borderRadius": "10px",
        "padding": "10px 12px",
        "margin": "6px",
        "textAlign": "center",
        "minWidth": "140px",
        "boxShadow": "0 0 0 1px rgba(255,255,255,0.02), 0 6px 18px rgba(0,0,0,0.35)"
    })

def style_fig(fig, height=None, legend=True):
    fig.update_layout(
        paper_bgcolor=COLORS["bg_dark"],
        plot_bgcolor=COLORS["bg_dark"],
        font=dict(color=COLORS["text"]),
        margin=dict(l=50, r=30, t=50, b=40),
        hoverlabel=dict(bgcolor="rgba(17,26,43,0.95)", font=dict(color=COLORS["text"])),
        showlegend=legend,
        title_x=0.5,
        xaxis=dict(showgrid=False, zeroline=False),
        yaxis=dict(gridcolor="rgba(255,255,255,0.05)", zeroline=False)
    )
    if height:
        fig.update_layout(height=height)
    return fig

# =========================
# Your Original Plot Functions
# =========================
def fig_home_vs_away_win_diff_annotated(m: pd.DataFrame):
    m = m[m["result"].notna()]
    m = m[~m["result"].str.lower().isin(["no result","match abandoned"])]
    city_team = {
        "Chennai":"Chennai Super Kings","Mumbai":"Mumbai Indians",
        "Bangalore":"Royal Challengers Bangalore","Bengaluru":"Royal Challengers Bangalore",
        "Hyderabad":"Sunrisers Hyderabad","Delhi":"Delhi Capitals",
        "Kolkata":"Kolkata Knight Riders","Jaipur":"Rajasthan Royals",
        "Mohali":"Punjab Kings","Lucknow":"Lucknow Super Giants",
        "Ahmedabad":"Gujarat Titans","Pune":"Rising Pune Supergiant",
        "Kochi":"Kochi Tuskers Kerala","Visakhapatnam":"Deccan Chargers"
    }
    m["home_team"] = m["City"].map(city_team)
    teams = pd.unique(pd.concat([m["team1"], m["team2"]]).dropna())
    def wpct(team, df):
        p = ((df["team1"] == team) | (df["team2"] == team)).sum()
        return (df["winner"].eq(team).sum() / p * 100) if p else np.nan
    data = []
    for t in teams:
        home = m[m["home_team"] == t]
        away = m[((m["team1"] == t) | (m["team2"] == t)) & (m["home_team"] != t)]
        if len(home) and len(away):
            h, a = wpct(t, home), wpct(t, away)
            data.append((t, round(h,1), round(a,1), round(h-a,1)))
    if not data: return go.Figure()
    df = pd.DataFrame(data, columns=["team","home_win%","away_win%","diff"]).nlargest(10,"diff",keep="all")
    df["label"] = df.apply(lambda r: f"Home {r['home_win%']}% | Away {r['away_win%']}%", axis=1)
    fig = go.Figure(go.Bar(
        y=df["team"], x=df["diff"], orientation="h", text=df["label"],
        marker_color=df["diff"].apply(lambda x: "#4CAF50" if x>0 else "#FF5252"),
        textposition="outside", width=0.5
    ))
    fig.update_layout(title="Top Teams: Home − Away Win% (All Seasons)",
                      xaxis_title="Difference in Win%", yaxis_title=None,
                      height=400, bargap=0.05)
    return style_fig(fig, height=400)

def fig_venue_pitch_type_fixed(m: pd.DataFrame, d: pd.DataFrame, season=None):
    if season not in [None,"All"] and "season" in m.columns:
        mid = m.loc[m["season"] == season, ["match_id","venue"]]; label = f"{season}"
    else:
        mid = m[["match_id","venue"]]; label = "All Seasons"
    d = d[d["match_id"].isin(mid["match_id"])]
    if d.empty: return px.bar(title=f"Venue Pitch Type — {label} (no data)")
    per_match = (d.groupby("match_id", as_index=False)["total_runs"].sum().rename(columns={"total_runs":"match_total"}))
    per_match = per_match.merge(mid, on="match_id", how="left")
    vavg = (per_match.groupby("venue", as_index=False)["match_total"].mean().rename(columns={"match_total":"avg_runs"}))
    cutoff = 330.0
    vavg["pitch_type"] = np.where(vavg["avg_runs"] >= cutoff, "Batting", "Bowling")
    vavg = vavg.sort_values("avg_runs")
    fig = px.bar(vavg, x="avg_runs", y="venue", orientation="h", color="pitch_type",
                 color_discrete_map={"Batting":"#2ecc71","Bowling":"#e74c3c"},
                 title=f"Venue's Pitch Type by Avg Runs — {label} (cutoff: 330)",
                 text=vavg["avg_runs"].round(1))
    fig.update_traces(textposition="outside", width=0.4)
    fig.update_layout(xaxis_title="Avg runs per match", yaxis_title=None,
                      height=520, bargap=0.35, legend_title="")
    return style_fig(fig, height=520)

def fig_toss_decision_by_start_time(m: pd.DataFrame):
    if "result" in m.columns:
        m = m[m["result"].notna()]
        m = m[~m["result"].str.lower().isin(["no result","match abandoned"])]
    t = m["Time"].astype(str).str.strip().str.replace(r"\s+"," ", regex=True)
    m["start_24h"] = pd.to_datetime(t, format="%I:%M %p", errors="coerce").dt.strftime("%H:%M")
    m.loc[m["start_24h"].isna(), "start_24h"] = t
    m["Toss_Decision"] = m["Toss_Decision"].str.title()
    m = m[m["Toss_Decision"].isin(["Bat","Field"])]
    ct = (m.groupby(["start_24h","Toss_Decision"]).size()
            .unstack(fill_value=0).reindex(columns=["Bat","Field"], fill_value=0).reset_index())
    ct = ct.sort_values("start_24h")
    fig = px.bar(ct.melt(id_vars="start_24h", value_vars=["Bat","Field"],
                         var_name="Decision", value_name="Matches"),
                 x="start_24h", y="Matches", color="Decision",
                 barmode="group", text="Matches",
                 title="Toss Decision by Match Start Time (All Seasons)",
                 color_discrete_map={"Bat":"#1f77b4","Field":"#ff7f0e"})
    fig.update_traces(width=0.35, textposition="outside")
    fig.update_layout(xaxis_title="Start Time (24h)", yaxis_title="Match Count",
                      bargap=0.35, height=360, legend_title="")
    return style_fig(fig, height=360)

def fig_toss_vs_match_winner(m: pd.DataFrame):
    if "result" in m.columns:
        m = m[m["result"].notna()]
        m = m[~m["result"].str.lower().isin(["no result","match abandoned"])]
    m["same"] = (m["winner"] == m["toss_winner"]).map({True:"Toss = Match", False:"Toss ≠ Match"})
    df = m.groupby(["season","same"]).size().reset_index(name="matches")
    if df.empty: return px.bar(title="Toss Winners vs Match Winners by Season (no data)")
    total = df.groupby("season")["matches"].transform("sum")
    df["pct"] = (df["matches"] / total * 100).round(1)
    fig = px.bar(df, x="season", y="pct", color="same", text="pct",
                 title="Toss Winners vs Match Winners by Season",
                 color_discrete_map={"Toss = Match":"#2ecc71","Toss ≠ Match":"#e74c3c"})
    fig.update_traces(textposition="inside")
    fig.update_layout(xaxis_title="Season", yaxis_title="% of Matches",
                      bargap=0.3, height=420, legend_title="")
    return style_fig(fig, height=420)

def fig_umpire_disparity_wide_lbw(m: pd.DataFrame, d: pd.DataFrame, season=None):
    if season not in [None,"All"] and "season" in m.columns:
        mids = m.loc[m["season"] == season, "match_id"].unique(); label = f"{season}"
    else:
        mids = m["match_id"].unique(); label = "All Seasons"
    d = d[d["match_id"].isin(mids)]
    if d.empty: return px.bar(title=f"Umpire Decision Disparity — {label} (no data)")
    ext = d.get("extras_type"); dsk = d.get("dismissal_kind")
    is_wide = ext.astype(str).str.lower().eq("wide") if ext is not None else pd.Series(False, index=d.index)
    is_lbw  = dsk.astype(str).str.lower().str.contains("lbw", na=False) if dsk is not None else pd.Series(False, index=d.index)
    w_for  = d.loc[is_wide].groupby("batting_team").size().rename("wides_for")
    w_agn  = d.loc[is_wide].groupby("bowling_team").size().rename("wides_against")
    l_for  = d.loc[is_lbw ].groupby("bowling_team").size().rename("lbw_for")
    l_agn  = d.loc[is_lbw ].groupby("batting_team").size().rename("lbw_against")
    df = pd.concat([w_for,w_agn,l_for,l_agn], axis=1).fillna(0).astype(int).reset_index().rename(columns={"index":"team"})
    if "team" not in df.columns: df = df.rename(columns={df.columns[0]:"team"})
    df["for"] = df[["wides_for","lbw_for"]].sum(axis=1)
    df["against"] = df[["wides_against","lbw_against"]].sum(axis=1)
    df["disparity"] = df["for"] - df["against"]
    denom = (df["for"] + df["against"]).replace(0, np.nan)
    df["disparity_pct"] = (df["disparity"] / denom * 100).round(1)
    df = df.sort_values("disparity")
    df["sign"] = np.where(df["disparity"]>=0, "Positive", "Negative")
    df["label"] = df.apply(lambda r: f"{int(r['disparity'])} ({'' if pd.isna(r['disparity_pct']) else r['disparity_pct']}%)", axis=1)
    fig = px.bar(df, x="disparity", y="team", orientation="h", color="sign", text="label",
                 color_discrete_map={"Positive":"#2ecc71","Negative":"#e74c3c"},
                 title=f"Umpire Decision Disparity (Wide→Bat, LBW→Bowl) — {label}")
    fig.update_traces(textposition="outside", width=0.4)
    fig.update_layout(xaxis_title="Disparity = In Favour − Against (counts)", yaxis_title=None,
                      height=560, bargap=0.35, legend_title="")
    return style_fig(fig, height=560)

# =========================
# Tab 1: Overall Overview (from your layout_tab0)
# =========================
def layout_tab1():
    m = matches_6.copy()
    d = deliveries_6.copy()
    vcoords = venue_coords.copy()

    # Calculate KPIs
    total_matches = len(m)
    total_runs = int(d["total_runs"].sum())
    total_wickets = int(d["is_wicket"].sum())
    avg_runs = round(total_runs / total_matches, 1) if total_matches else 0
    avg_wickets = round(total_wickets / total_matches, 2) if total_matches else 0
    unique_venues = m["venue"].nunique()

    # Most trophies
    finals = m[m["result"].str.contains("Final", case=False, na=False)]
    titles = finals["winner"].value_counts()
    if not titles.empty:
        max_titles = titles.max()
        winners_with_max = titles[titles == max_titles].index.tolist()
        winners_abbr = [get_team_abbr(t) for t in winners_with_max]
        most_trophies = " & ".join(winners_abbr) + f" ({max_titles})"
    else:
        most_trophies = "N/A"

    # Most POTM
    potm_counts = m["player_of_match"].value_counts()
    most_potm = f"{potm_counts.index[0]} — {int(potm_counts.iloc[0])}" if not potm_counts.empty else "N/A"

    # Best team by win percentage
    wins = m["winner"].value_counts().rename_axis("team").reset_index(name="wins")
    played = pd.concat([m[["team1"]].rename(columns={"team1": "team"}), 
                        m[["team2"]].rename(columns={"team2": "team"})])
    played = played["team"].value_counts().rename_axis("team").reset_index(name="matches_played")
    team_perf = wins.merge(played, on="team", how="right").fillna(0)
    team_perf["win_pct"] = np.where(team_perf["matches_played"] > 0,
                                    team_perf["wins"] / team_perf["matches_played"] * 100, 0)
    team_perf = team_perf.sort_values("win_pct", ascending=False)
    best_team = team_perf.iloc[0]["team"] if not team_perf.empty else "N/A"
    best_team_pct = round(team_perf.iloc[0]["win_pct"], 1) if not team_perf.empty else 0
    best_team_display = f"{get_team_abbr(best_team)} — {best_team_pct}%"

    # Highest match total
    if not d.empty:
        match_totals = d.groupby("match_id")["total_runs"].sum().reset_index()
        highest_match = match_totals.sort_values("total_runs", ascending=False).head(1)
        if not highest_match.empty:
            hm_id = int(highest_match.iloc[0]["match_id"])
            hm_runs = int(highest_match.iloc[0]["total_runs"])
            row = m[m["match_id"] == hm_id]
            if not row.empty:
                hi_teams = f"{get_team_abbr(row.iloc[0]['team1'])} vs {get_team_abbr(row.iloc[0]['team2'])}"
            else:
                hi_teams = str(hm_id)
            highest_match_display = f"{hi_teams} — {hm_runs} runs"
        else:
            highest_match_display = "N/A"
    else:
        highest_match_display = "N/A"

    # KPI Row
    kpi_row = html.Div([
        kpi_card_unified("Total Matches", total_matches, COLORS["accent"]),
        kpi_card_unified("Total Runs", f"{total_runs:,}", "#00e676"),
        kpi_card_unified("Total Wickets", f"{total_wickets:,}", "#f44336"),
        kpi_card_unified("Avg Runs / Match", avg_runs, "#29b6f6"),
        kpi_card_unified("Avg Wickets / Match", avg_wickets, "#e91e63"),
        kpi_card_unified("Venues", unique_venues, "#ce93d8"),
        kpi_card_unified("Most Trophies", most_trophies, COLORS["accent"]),
        kpi_card_unified("Most POTM", most_potm, "#ffd166"),
        kpi_card_unified("Best Team (Win%)", best_team_display, pick_color_for_teamname(best_team)),
        kpi_card_unified("Highest Match Total", highest_match_display, "#ffb703"),
    ], style={"display": "flex", "justifyContent": "center", "flexWrap": "wrap"})

    # Avg Runs & Wickets per Season
    season_map = m.set_index("match_id")["season"].to_dict()
    d_map = d.copy()
    d_map["season"] = d_map["match_id"].map(season_map)
    match_stats = d_map.groupby(["match_id", "season"], dropna=False)[["total_runs", "is_wicket"]].sum().reset_index()
    season_stats = match_stats.groupby("season")[["total_runs", "is_wicket"]].mean().reset_index().sort_values("season")

    fig_rw = go.Figure()
    fig_rw.add_trace(go.Scatter(x=season_stats["season"], y=season_stats["total_runs"],
                                name="Avg Runs per Match", mode="lines+markers",
                                line=dict(color="#00b4d8", width=3)))
    fig_rw.add_trace(go.Scatter(x=season_stats["season"], y=season_stats["is_wicket"],
                                name="Avg Wickets per Match", mode="lines+markers",
                                line=dict(color="#f94144", width=3), yaxis="y2"))
    fig_rw = style_fig(fig_rw, height=350)

    # Heatmaps
    ball_counts = d_map.groupby(["season", "over"]).size().rename("balls").reset_index()
    runs_sum = d_map.groupby(["season", "over"])["batsman_runs"].sum().rename("runs").reset_index()
    wickets_sum = d_map.groupby(["season", "over"])["is_wicket"].sum().rename("wickets").reset_index()
    
    heat = runs_sum.merge(ball_counts, on=["season", "over"], how="left")
    heat["runs_per_over"] = heat["runs"] / heat["balls"] * 6
    heat2 = wickets_sum.merge(ball_counts, on=["season", "over"], how="left")
    heat2["wickets_per_over"] = heat2["wickets"] / heat2["balls"] * 6
    
    pivot_runs = heat.pivot(index="season", columns="over", values="runs_per_over").sort_index(ascending=True).fillna(0)
    pivot_wk = heat2.pivot(index="season", columns="over", values="wickets_per_over").sort_index(ascending=True).fillna(0)

    fig_bat = px.imshow(pivot_runs, color_continuous_scale="YlOrRd",
                        labels=dict(x="Over", y="Season", color="Avg Runs / Over"),
                        aspect="auto", title="Batting Intensity — Avg Runs per Over")
    fig_bat = style_fig(fig_bat, height=360)

    fig_bowl = px.imshow(pivot_wk, color_continuous_scale="PuBuGn",
                         labels=dict(x="Over", y="Season", color="Avg Wkts / Over"),
                         aspect="auto", title="Bowling Intensity — Avg Wickets per Over")
    fig_bowl = style_fig(fig_bowl, height=360)

    # Top 10 batters and bowlers
    batter_team = d_map.groupby(["batter", "batting_team"]).size().reset_index(name="count") \
                   .sort_values(["batter", "count"], ascending=[True, False]) \
                   .drop_duplicates("batter").set_index("batter")["batting_team"].to_dict()
    bowler_team = d_map.groupby(["bowler", "bowling_team"]).size().reset_index(name="count") \
                   .sort_values(["bowler", "count"], ascending=[True, False]) \
                   .drop_duplicates("bowler").set_index("bowler")["bowling_team"].to_dict()

    top_bat = d_map.groupby("batter")["batsman_runs"].sum().reset_index().sort_values("batsman_runs", ascending=False).head(10)
    top_bat["team"] = top_bat["batter"].map(batter_team).fillna("NA")
    top_bat["abbr"] = top_bat["team"].apply(get_team_abbr)
    top_bat["color"] = top_bat["team"].apply(pick_color_for_teamname).fillna("#9e9e9e")
    
    fig_bat_top = go.Figure(go.Bar(
        x=top_bat["batsman_runs"][::-1],
        y=(top_bat["batter"] + " — " + top_bat["abbr"])[::-1],
        orientation="h", marker_color=top_bat["color"][::-1],
        text=top_bat["batsman_runs"][::-1], textposition="outside"
    ))
    fig_bat_top = style_fig(fig_bat_top, height=360)

    top_bowl = d_map[d_map["is_wicket"] == 1].groupby("bowler").size().reset_index(name="wickets").sort_values("wickets", ascending=False).head(10)
    top_bowl["team"] = top_bowl["bowler"].map(bowler_team).fillna("NA")
    top_bowl["abbr"] = top_bowl["team"].apply(get_team_abbr)
    top_bowl["color"] = top_bowl["team"].apply(pick_color_for_teamname).fillna("#9e9e9e")
    
    fig_bowl_top = go.Figure(go.Bar(
        x=top_bowl["wickets"][::-1],
        y=(top_bowl["bowler"] + " — " + top_bowl["abbr"])[::-1],
        orientation="h", marker_color=top_bowl["color"][::-1],
        text=top_bowl["wickets"][::-1], textposition="outside"
    ))
    fig_bowl_top = style_fig(fig_bowl_top, height=360)

    # POTM Top 10
    potm = m["player_of_match"].value_counts().reset_index().head(10)
    potm.columns = ["player", "awards"]
    player_team = d_map.groupby(["batter", "batting_team"]).size().reset_index(name="count") \
                   .sort_values(["batter", "count"], ascending=[True, False]) \
                   .drop_duplicates("batter").set_index("batter")["batting_team"].to_dict()
    potm["team"] = potm["player"].map(player_team).fillna("NA")
    potm["abbr"] = potm["team"].apply(get_team_abbr)
    potm["color"] = potm["team"].apply(pick_color_for_teamname).fillna("#9e9e9e")
    potm = potm.sort_values("awards", ascending=False)
    
    fig_potm = go.Figure(go.Bar(
        x=potm["awards"][::-1],
        y=(potm["player"] + " — " + potm["abbr"])[::-1],
        orientation="h", marker_color=potm["color"][::-1],
        text=potm["awards"][::-1], textposition="outside"
    ))
    fig_potm = style_fig(fig_potm, height=360)

    # Venue Map
    v_counts = m["venue"].value_counts().reset_index()
    v_counts.columns = ["venue", "matches"]
    vmap = pd.merge(v_counts, vcoords, on="venue", how="left")
    
    def venue_color(home):
        if pd.isna(home): return "#9e9e9e"
        if home in REGION_COLORS: return REGION_COLORS[home]
        return pick_color_for_teamname(home)
    
    vmap["color"] = vmap["home"].apply(venue_color)
    
    fig_map = go.Figure()
    for _, row in vmap.iterrows():
        fig_map.add_trace(go.Scattergeo(
            lon=[row["lon"]], lat=[row["lat"]],
            text=f"{row['venue']}<br>{int(row['matches'])} matches<br>Home: {row['home']}",
            name=row["venue"],
            marker=dict(size=max(6, row["matches"]**0.5 * 2.5), color=row["color"],
                        line=dict(width=0.6, color="white"), opacity=0.9),
            hoverinfo="text"
        ))
    
    fig_map.update_layout(
        title="IPL Venues", height=500, showlegend=False,
        geo=dict(scope="asia", projection_type="mercator",
                 center=dict(lat=22, lon=78),
                 lataxis=dict(range=[5, 35]), lonaxis=dict(range=[65, 90]),
                 showland=True, landcolor="rgb(15,25,40)",
                 showocean=True, oceancolor="rgb(10,15,25)",
                 showcountries=True, countrycolor="rgba(255,255,255,0.3)",
                 showframe=False),
        plot_bgcolor=COLORS["bg_dark"], paper_bgcolor=COLORS["bg_dark"], font_color="white", title_x=0.5
    )

    # Boundary % per Season
    boundaries = d_map[d_map["batsman_runs"].isin([4, 6])].groupby("season")["batsman_runs"].count().reset_index(name="boundaries")
    total_balls = d_map.groupby("season").size().reset_index(name="balls")
    boundary_rate = boundaries.merge(total_balls, on="season", how="right").fillna(0)
    boundary_rate["boundary_pct"] = (boundary_rate["boundaries"] / boundary_rate["balls"]) * 100
    boundary_rate = boundary_rate.sort_values("season")
    
    fig_boundary = go.Figure(go.Scatter(
        x=boundary_rate["season"], y=boundary_rate["boundary_pct"], mode="lines+markers",
        line=dict(color="#ffd166", width=3), marker=dict(size=8, color="#ffd166", line=dict(width=1.2, color="white")),
        fill="tozeroy", fillcolor="rgba(255,209,102,0.15)",
        hovertemplate="<b>Season %{x}</b><br>Boundary %: %{y:.2f}%<extra></extra>"
    ))
    fig_boundary = style_fig(fig_boundary, height=300)

    # Dot Ball % per Season
    dot_balls = d_map[d_map["batsman_runs"] == 0].groupby("season").size().reset_index(name="dot_balls")
    total_balls2 = d_map.groupby("season").size().reset_index(name="total_balls")
    dot_rate = dot_balls.merge(total_balls2, on="season", how="right").fillna(0)
    dot_rate["dot_ball_pct"] = (dot_rate["dot_balls"] / dot_rate["total_balls"]) * 100
    dot_rate = dot_rate.sort_values("season")
    
    fig_dot = go.Figure(go.Scatter(
        x=dot_rate["season"], y=dot_rate["dot_ball_pct"], mode="lines+markers",
        line=dict(color="#00b4d8", width=3, shape="spline"),
        marker=dict(size=9, color="#00b4d8", line=dict(width=1.5, color="white")),
        fill="tozeroy", fillcolor="rgba(0,180,216,0.15)",
        hovertemplate="<b>Season %{x}</b><br>Dot Ball %: %{y:.2f}%<extra></extra>"
    ))
    fig_dot = style_fig(fig_dot, height=360)

    return html.Div([
        html.H2("🏟️ IPL — Overall Overview", style={"textAlign": "center", "color": COLORS["accent"], "marginBottom": "10px"}),
        kpi_row,
        html.Div([dcc.Graph(figure=fig_rw)], style={"marginTop": "8px"}),
        html.Div([
            html.Div(dcc.Graph(figure=fig_bat), style={"width": "49%", "display": "inline-block"}),
            html.Div(dcc.Graph(figure=fig_bowl), style={"width": "49%", "display": "inline-block", "marginLeft": "2%"}),
        ], style={"marginTop": "6px"}),
        html.Div([
            html.Div(dcc.Graph(figure=fig_bat_top), style={"width": "49%", "display": "inline-block"}),
            html.Div(dcc.Graph(figure=fig_bowl_top), style={"width": "49%", "display": "inline-block", "marginLeft": "2%"}),
        ], style={"marginTop": "6px"}),
        html.Div([dcc.Graph(figure=fig_potm)], style={"marginTop": "8px"}),
        html.Div([dcc.Graph(figure=fig_map)], style={"marginTop": "8px"}),
        html.Div([
            html.Div(dcc.Graph(figure=fig_boundary), style={"width": "49%", "display": "inline-block"}),
            html.Div(dcc.Graph(figure=fig_dot), style={"width": "49%", "display": "inline-block", "marginLeft": "2%"}),
        ], style={"marginTop": "6px", "marginBottom": "40px"}),
    ], style={"backgroundColor": COLORS["bg_dark"], "color": COLORS["text"], "padding": "18px"})

# =========================
# Tab 2: Combined Analytics
# =========================
def layout_tab2():
    return html.Div([
        html.H2("Combined Analytics", style={"color": COLORS["accent"], "textAlign": "center", "marginBottom": "10px"}),
        html.Div(style={"display":"grid","gridTemplateColumns":"1fr 1fr","gap":"12px","marginTop":"12px"}, children=[
            dcc.Graph(id="fig-home-away", figure=fig_home_vs_away_win_diff_annotated(matches_6)),
            dcc.Graph(id="fig-toss-start", figure=fig_toss_decision_by_start_time(matches_6)),
        ]),
        html.Div(style={"marginTop":"12px"}, children=[
            dcc.Graph(id="fig-toss-vs-match", figure=fig_toss_vs_match_winner(matches_6))
        ]),
        html.Hr(style={"borderColor": "rgba(255,255,255,0.08)"}),
        html.Div(style={"display":"flex","gap":"10px","alignItems":"center","marginTop":"10px"}, children=[
            html.Label("Season (Pitch Types):", style={"minWidth":"150px", "color": COLORS["text"]}),
            dcc.Dropdown(
                id="dd-season-pitch",
                options=([{"label":"All Seasons","value":"All"}] + [{"label":str(s),"value":s} for s in SEASONS]) or [{"label":"All Seasons","value":"All"}],
                value="All", clearable=False, style={"width":"220px","color":"#000"}
            ),
        ]),
        dcc.Graph(id="fig-pitch"),
        html.Div(style={"display":"flex","gap":"10px","alignItems":"center","marginTop":"10px"}, children=[
            html.Label("Season (Umpire Disparity):", style={"minWidth":"150px", "color": COLORS["text"]}),
            dcc.Dropdown(
                id="dd-season-umpire",
                options=([{"label":"All Seasons","value":"All"}] + [{"label":str(s),"value":s} for s in SEASONS]) or [{"label":"All Seasons","value":"All"}],
                value="All", clearable=False, style={"width":"220px","color":"#000"}
            ),
        ]),
        dcc.Graph(id="fig-umpire")
    ], style={"backgroundColor": COLORS["bg_dark"], "color": COLORS["text"], "padding": "18px"})

# =========================
# Tab 3: Player Explorer
# =========================
def layout_tab3():
    # Get unique players
    players = sorted(pd.unique(pd.concat([deliveries_6["batter"], deliveries_6["bowler"]]).dropna()))
    
    return html.Div([
        html.H2("Player Explorer", style={"color": COLORS["accent"], "textAlign": "center", "marginBottom": "10px"}),
        html.Div([
            html.Div([
                html.Label("Select Player:", style={"marginRight": "10px", "color": COLORS["text"]}),
                dcc.Dropdown(
                    id="player-dropdown",
                    options=[{"label": p, "value": p} for p in players],
                    value=players[0] if players else None,
                    clearable=False,
                    style={"width": "300px", "color": "#000"}
                )
            ], style={"display": "flex", "justifyContent": "center", "alignItems": "center", "marginBottom": "20px"}),
        ]),
        html.Div(id="player-kpi-cards"),
        html.Div(id="player-charts")
    ], style={"backgroundColor": COLORS["bg_dark"], "color": COLORS["text"], "padding": "18px"})

def register_callbacks_tab3(app):
    @app.callback(
        [Output("player-kpi-cards", "children"),
         Output("player-charts", "children")],
        [Input("player-dropdown", "value")]
    )
    def update_player_explorer(player):
        if not player:
            return [], []
            
        # Filter data for selected player
        player_batting = deliveries_6[deliveries_6["batter"] == player]
        player_bowling = deliveries_6[deliveries_6["bowler"] == player]
        
        # Calculate player KPIs
        total_runs = int(player_batting["batsman_runs"].sum()) if not player_batting.empty else 0
        total_wickets = int(player_bowling["is_wicket"].sum()) if not player_bowling.empty else 0
        
        # Batting stats
        batting_innings = player_batting["match_id"].nunique() if not player_batting.empty else 0
        batting_balls = len(player_batting[~player_batting["extras_type"].isin(["wides", "noballs"])]) if not player_batting.empty else 0
        batting_strike_rate = round((total_runs / batting_balls * 100), 2) if batting_balls > 0 else 0
        
        # Bowling stats
        bowling_innings = player_bowling["match_id"].nunique() if not player_bowling.empty else 0
        bowling_balls = len(player_bowling[~player_bowling["extras_type"].isin(["wides", "noballs"])]) if not player_bowling.empty else 0
        bowling_runs = int(player_bowling["total_runs"].sum()) if not player_bowling.empty else 0
        bowling_economy = round((bowling_runs / (bowling_balls / 6)), 2) if bowling_balls > 0 else 0
        
        # KPI Cards
        kpi_cards = html.Div([
            kpi_card_unified("Total Runs", total_runs, "#00e676"),
            kpi_card_unified("Total Wickets", total_wickets, "#f44336"),
            kpi_card_unified("Batting SR", batting_strike_rate),
            kpi_card_unified("Bowling Economy", bowling_economy),
            kpi_card_unified("Batting Innings", batting_innings),
            kpi_card_unified("Bowling Innings", bowling_innings),
        ], style={"display": "flex", "justifyContent": "center", "flexWrap": "wrap", "marginBottom": "20px"})
        
        # Player performance by season
        player_with_season = deliveries_6.merge(matches_6[["match_id", "season"]], on="match_id")
        
        # Batting by season
        batting_by_season = player_with_season[player_with_season["batter"] == player].groupby("season")["batsman_runs"].sum().reset_index()
        fig_batting_season = px.line(batting_by_season, x="season", y="batsman_runs", 
                                    title=f"{player} - Runs by Season", markers=True)
        fig_batting_season = style_fig(fig_batting_season, height=400)
        
        # Bowling by season
        bowling_by_season = player_with_season[player_with_season["bowler"] == player].groupby("season")["is_wicket"].sum().reset_index()
        fig_bowling_season = px.line(bowling_by_season, x="season", y="is_wicket", 
                                    title=f"{player} - Wickets by Season", markers=True)
        fig_bowling_season = style_fig(fig_bowling_season, height=400)
        
        # Dismissal types
        dismissals = player_with_season[(player_with_season["player_dismissed"] == player) & 
                                       (player_with_season["is_wicket"] == 1)]
        dismissal_counts = dismissals["dismissal_kind"].value_counts().reset_index()
        dismissal_counts.columns = ["dismissal_kind", "count"]
        
        fig_dismissals = px.pie(dismissal_counts, values="count", names="dismissal_kind", 
                               title=f"{player} - Dismissal Types")
        fig_dismissals = style_fig(fig_dismissals, height=400)
        
        charts = html.Div([
            dbc.Row([
                dbc.Col(dcc.Graph(figure=fig_batting_season), md=6),
                dbc.Col(dcc.Graph(figure=fig_bowling_season), md=6),
            ]),
            dbc.Row([
                dbc.Col(dcc.Graph(figure=fig_dismissals), md=6),
            ])
        ])
        
        return kpi_cards, charts

# =========================
# Tab 4: Head to Head
# =========================
def layout_tab4():
    # Get unique teams
    teams = sorted(pd.unique(pd.concat([matches_6["team1"], matches_6["team2"]]).dropna()))
    
    return html.Div([
        html.H2("Head to Head", style={"color": COLORS["accent"], "textAlign": "center", "marginBottom": "10px"}),
        html.Div([
            html.Div([
                html.Label("Team 1:", style={"marginRight": "10px", "color": COLORS["text"]}),
                dcc.Dropdown(
                    id="team1-dropdown",
                    options=[{"label": t, "value": t} for t in teams],
                    value=teams[0] if teams else None,
                    clearable=False,
                    style={"width": "250px", "color": "#000"}
                )
            ], style={"display": "inline-block", "marginRight": "20px"}),
            html.Div([
                html.Label("Team 2:", style={"marginRight": "10px", "color": COLORS["text"]}),
                dcc.Dropdown(
                    id="team2-dropdown",
                    options=[{"label": t, "value": t} for t in teams],
                    value=teams[1] if len(teams) > 1 else None,
                    clearable=False,
                    style={"width": "250px", "color": "#000"}
                )
            ], style={"display": "inline-block"}),
        ], style={"display": "flex", "justifyContent": "center", "alignItems": "center", "marginBottom": "20px"}),
        html.Div(id="h2h-kpi-cards"),
        html.Div(id="h2h-charts")
    ], style={"backgroundColor": COLORS["bg_dark"], "color": COLORS["text"], "padding": "18px"})

def register_callbacks_tab4(app):
    @app.callback(
        [Output("h2h-kpi-cards", "children"),
         Output("h2h-charts", "children")],
        [Input("team1-dropdown", "value"),
         Input("team2-dropdown", "value")]
    )
    def update_head_to_head(team1, team2):
        if not team1 or not team2 or team1 == team2:
            return [], []
            
        # Filter matches between the two teams
        h2h_matches = matches_6[((matches_6["team1"] == team1) & (matches_6["team2"] == team2)) |
                               ((matches_6["team1"] == team2) & (matches_6["team2"] == team1))]
        
        # Calculate H2H stats
        total_matches = len(h2h_matches)
        team1_wins = len(h2h_matches[h2h_matches["winner"] == team1])
        team2_wins = len(h2h_matches[h2h_matches["winner"] == team2])
        ties = len(h2h_matches[h2h_matches["winner"].isna() | (h2h_matches["result"] == "tie")])
        
        team1_win_pct = round((team1_wins / total_matches * 100), 1) if total_matches > 0 else 0
        team2_win_pct = round((team2_wins / total_matches * 100), 1) if total_matches > 0 else 0
        
        # KPI Cards
        kpi_cards = html.Div([
            kpi_card_unified("Total Matches", total_matches, COLORS["accent"]),
            kpi_card_unified(f"{get_team_abbr(team1)} Wins", f"{team1_wins} ({team1_win_pct}%)", pick_color_for_teamname(team1)),
            kpi_card_unified(f"{get_team_abbr(team2)} Wins", f"{team2_wins} ({team2_win_pct}%)", pick_color_for_teamname(team2)),
            kpi_card_unified("Ties/No Results", ties, "#fdd835"),
        ], style={"display": "flex", "justifyContent": "center", "flexWrap": "wrap", "marginBottom": "20px"})
        
        # H2H by season
        h2h_by_season = h2h_matches.groupby("season").agg({
            "winner": lambda x: ((x == team1).sum(), (x == team2).sum())
        }).reset_index()
        
        # Prepare data for plotting
        seasons = []
        team1_wins_by_season = []
        team2_wins_by_season = []
        
        for _, row in h2h_by_season.iterrows():
            seasons.append(row["season"])
            wins_data = row["winner"]
            team1_wins_by_season.append(wins_data[0])
            team2_wins_by_season.append(wins_data[1])
        
        fig_h2h_season = go.Figure()
        fig_h2h_season.add_trace(go.Bar(
            x=seasons,
            y=team1_wins_by_season,
            name=get_team_abbr(team1),
            marker_color=pick_color_for_teamname(team1)
        ))
        fig_h2h_season.add_trace(go.Bar(
            x=seasons,
            y=team2_wins_by_season,
            name=get_team_abbr(team2),
            marker_color=pick_color_for_teamname(team2)
        ))
        fig_h2h_season.update_layout(barmode="group", title=f"{get_team_abbr(team1)} vs {get_team_abbr(team2)} - Wins by Season")
        fig_h2h_season = style_fig(fig_h2h_season, height=400)
        
        # Win comparison chart
        fig_win_comparison = go.Figure(go.Bar(
            x=[get_team_abbr(team1), get_team_abbr(team2)],
            y=[team1_wins, team2_wins],
            marker_color=[pick_color_for_teamname(team1), pick_color_for_teamname(team2)]
        ))
        fig_win_comparison = style_fig(fig_win_comparison, height=400)
        fig_win_comparison.update_layout(title="Win Comparison")
        
        charts = html.Div([
            dbc.Row([
                dbc.Col(dcc.Graph(figure=fig_h2h_season), md=6),
                dbc.Col(dcc.Graph(figure=fig_win_comparison), md=6),
            ])
        ])
        
        return kpi_cards, charts

# =========================
# Main App
# =========================
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP], suppress_callback_exceptions=True)
server = app.server

# Disable caching for simplicity
class _NoCache:
    def memoize(self, *args, **kwargs):
        def _decorator(f):
            return f
        return _decorator
cache = _NoCache()

# App layout
app.layout = html.Div(
    style={"backgroundColor": COLORS["bg_dark"], "minHeight": "100vh"},
    children=[
        html.Div(
            style={"maxWidth": "1400px", "margin": "0 auto", "padding": "20px"},
            children=[
                html.H1("IPL Analytics Dashboard", 
                       style={"color": COLORS["accent"], "textAlign": "center", "marginBottom": "30px"}),
                
                dcc.Tabs(id="main-tabs", value="tab-1", children=[
                    dcc.Tab(label="Overall Overview", value="tab-1",
                           style={"backgroundColor": "#0e172a", "color": "#cbd5e1"},
                           selected_style={"backgroundColor": "#111a2b", "color": "#ffffff", 
                                         "borderBottom": f"3px solid {COLORS['accent']}"}),
                    dcc.Tab(label="Combined Analytics", value="tab-2",
                           style={"backgroundColor": "#0e172a", "color": "#cbd5e1"},
                           selected_style={"backgroundColor": "#111a2b", "color": "#ffffff",
                                         "borderBottom": f"3px solid {COLORS['accent']}"}),
                    dcc.Tab(label="Player Explorer", value="tab-3",
                           style={"backgroundColor": "#0e172a", "color": "#cbd5e1"},
                           selected_style={"backgroundColor": "#111a2b", "color": "#ffffff",
                                         "borderBottom": f"3px solid {COLORS['accent']}"}),
                    dcc.Tab(label="Head to Head", value="tab-4",
                           style={"backgroundColor": "#0e172a", "color": "#cbd5e1"},
                           selected_style={"backgroundColor": "#111a2b", "color": "#ffffff",
                                         "borderBottom": f"3px solid {COLORS['accent']}"}),
                ]),
                
                html.Div(id="tab-content", style={"marginTop": "20px"})
            ]
        )
    ]
)

@app.callback(
    Output("tab-content", "children"),
    [Input("main-tabs", "value")]
)
def render_tab_content(tab):
    if tab == "tab-1":
        return layout_tab1()
    elif tab == "tab-2":
        return layout_tab2()
    elif tab == "tab-3":
        return layout_tab3()
    elif tab == "tab-4":
        return layout_tab4()
    else:
        return html.Div("Select a tab")

# Callbacks for Combined Analytics tab
@app.callback(Output("fig-pitch","figure"), Input("dd-season-pitch","value"))
def _update_pitch(season):
    return fig_venue_pitch_type_fixed(matches_6, deliveries_6, season=season)

@app.callback(Output("fig-umpire","figure"), Input("dd-season-umpire","value"))
def _update_umpire(season):
    return fig_umpire_disparity_wide_lbw(matches_6, deliveries_6, season=season)

# Register other callbacks
register_callbacks_tab3(app)
register_callbacks_tab4(app)

if __name__ == "__main__":
    app.run(debug=True, port=8057)

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
File ~\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pandas\core\indexes\base.py:3805, in Index.get_loc(
    self=Index(['match_id', 'season', 'Match_No', 'City',... 'Method', 'ump1', 'ump2'],
      dtype='object'),
    key='player_of_match'
)
   3804 try:
-> 3805     return self._engine.get_loc(casted_key)
        casted_key = 'player_of_match'
        self = Index(['match_id', 'season', 'Match_No', 'City', 'date', 'Time', 'Match_Type',
       'Player_Of_Match', 'venue', 'team1', 'team2', 'toss_winner',
       'Toss_Decision', 'winner', 'result', 'Result_Margin', 'Target_Runs',
       'Target_Overs', 'Super_Over', 'Method', 'ump1', 'ump2'],
      dtype='object')
   3806 except KeyError as err:

File index.pyx:167, in pandas._libs.index.IndexEngine.get_loc()

